In [ ]:
import json
from statistics import mean

# Load the timing results JSON file
with open("sae_bench_eval_timing_results.json", "r") as f:
    timing_results = json.load(f)

# Initialize variables to track totals
total_avg_time = 0
total_setup_time = 0

# Prepare LaTeX table content
latex_table = r"""
\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|}
\hline
Evaluation Type & Avg Time per SAE (min) & Setup Time (min) \\ \hline
"""

# Process each evaluation type
for eval_type, eval_data in timing_results.items():
    iterations = eval_data.get("iterations", [])
    if not iterations:
        continue
        
    # Exclude the first iteration for average SAE time
    subsequent_iterations = iterations[1:]
    avg_time_per_sae = (
        mean(iteration["total_iteration_time"] for iteration in subsequent_iterations) / 60
        if subsequent_iterations
        else 0
    )
    
    # Setup time for the first iteration (total time minus average iteration time)
    first_iteration = iterations[0]
    avg_iteration_time = avg_time_per_sae * 60  # Convert back to seconds for subtraction
    setup_time = (first_iteration["total_iteration_time"] - avg_iteration_time) / 60
    setup_time = max(setup_time, 0)  # Ensure no negative setup time
    
    # Add to totals
    total_avg_time += avg_time_per_sae
    total_setup_time += setup_time
    
    # Append to LaTeX table
    latex_table += (
        f"{eval_type} & {round(avg_time_per_sae)} & {round(setup_time)} \\\\ \\hline\n"
    )

# Add the total row
latex_table += (
    f"\\textbf{{Total}} & \\textbf{{{round(total_avg_time)}}} & "
    f"\\textbf{{{round(total_setup_time)}}} \\\\ \\hline\n"
)

# Complete the LaTeX table
latex_table += r"""
\end{tabular}
\caption{Timing results for evaluations, rounded to the nearest minute.}
\label{tab:eval_timing}
\end{table}
"""

# Save the LaTeX table to a .tex file
with open("timing_results_table.tex", "w") as f:
    f.write(latex_table)

print("LaTeX table has been saved to 'timing_results_table.tex'.")